In [1]:
#Please upload kaggle.json before running this
# Make a Directory and the move the uploaded kaggle json to hidden kaggle folder
! mkdir -p /root/.kaggle
! cp kaggle.json /root/.kaggle 
import kaggle

In [0]:
#check the tensorflow version and ensure its 2.x+
%tensorflow_version 2.x

In [3]:
!kaggle competitions download -c dogs-vs-cats

!unzip -q train.zip

 94% 256M/271M [00:03<00:00, 58.7MB/s]
100% 271M/271M [00:04<00:00, 70.6MB/s]
sampleSubmission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm import tqdm_notebook

In [0]:
#Image.open("./train/cat.7751.jpg")

In [0]:
#Check shape of above image
#cv2.imread("./train/cat.7751.jpg").shape


In [45]:
all_image_name = os.listdir("./train")
df = pd.DataFrame(all_image_name, columns=["image_name"])
df["final_image_name"] = df.image_name
df.head()


,image_name,final_image_name
0,cat.3754.jpg,cat.3754.jpg
1,dog.7461.jpg,dog.7461.jpg
2,dog.10169.jpg,dog.10169.jpg
3,dog.3897.jpg,dog.3897.jpg
4,cat.686.jpg,cat.686.jpg


In [48]:
#Above since image names has dot (.) in it so lets split it
df2 = df.image_name.str.split(".", expand=True)
df2["final_image_name"] = df.final_image_name
del df2[1]
del df2[2]
df2.head()

,0,final_image_name
0,cat,cat.3754.jpg
1,dog,dog.7461.jpg
2,dog,dog.10169.jpg
3,dog,dog.3897.jpg
4,cat,cat.686.jpg


In [8]:
classes = df2[0].unique().tolist()
classes

['cat', 'dog']

In [54]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train, y_test = train_test_split(df2.final_image_name, df2[0], test_size=0.2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((20000,), (20000,), (5000,), (5000,))

In [0]:
x_train, x_val, y_train, y_val= train_test_split(x_train, y_train, test_size=0.2)

In [56]:
x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape

((16000,), (4000,), (5000,), (16000,), (4000,), (5000,))

In [58]:
#Sample data
# df2.sample(frac=0.2).shape

(5000, 2)

In [0]:
#validation_df.size

In [12]:
df.size

25000

In [13]:
!ls -ltr /content
!mkdir data

total 834740
drwxr-xr-x 2 root root    798720 Sep 20  2013 train
drwxr-xr-x 1 root root      4096 Mar 18 16:23 sample_data
-rw-r--r-- 1 root root        69 Apr  5 12:15 kaggle.json
-rw-r--r-- 1 root root     88903 Apr  5 12:29 sampleSubmission.csv
-rw-r--r-- 1 root root 569546721 Apr  5 12:29 train.zip
-rw-r--r-- 1 root root 284321224 Apr  5 13:41 test1.zip


In [0]:
!mkdir validation_data

In [0]:
#Create the image data generator structure
for _class in classes:
    os.mkdir("data/" + _class)
    os.mkdir("validation_data/" + _class)

In [0]:
SRC_FOLDER = "./train/"
CAT_FOLDER = "./data/cat/"
DOG_FODLER = "./data/dog/"
V_FODLER = "./validation_data/"
V_CAT_FOLDER = "./validation_data/cat/"
V_DOG_FODLER = "./validation_data/dog/"


In [0]:
for image in os.listdir(SRC_FOLDER):
    if image.lower().startswith("cat"):
        shutil.copy(SRC_FOLDER + image, CAT_FOLDER + image)
    else:
        shutil.copy(SRC_FOLDER + image, DOG_FODLER + image)

In [60]:
# for image in x_val:
#   if image.lower().startswith("cat"):
#     shutil.copy(SRC_FOLDER + image, V_CAT_FOLDER + image)
#   else:
#     shutil.copy(SRC_FOLDER + image, V_DOG_FODLER + image) 

cat.10366.jpg


In [0]:
counter = 0
for image in os.listdir(SRC_FOLDER):
    counter += 1
    if (counter % 5) == 0:
        if image.lower().startswith("cat"):
            shutil.copy(SRC_FOLDER + image, V_CAT_FOLDER + image)
        else:
            shutil.copy(SRC_FOLDER + image, V_DOG_FODLER + image) 
    else:
        print(" ignored this row")

In [19]:
!ls -ltr /content/data/dog | wc

  12501  112502  688903


In [20]:
#Lets see sample dog size
!ls -ltr /content/validation_data/cat | wc


   2492   22421  137309


In [0]:
# Libraries to import for Image Processing now
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


In [0]:
# Create a dataGenerator object
dataGenerator = ImageDataGenerator(rotation_range=40, horizontal_flip=True, zoom_range=0.2, rescale=1/255)

In [0]:
#Import libraries for Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation, Flatten, Dropout, MaxPooling2D, Conv2D
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD

In [0]:
model = Sequential()
#chunk-1
model.add(Conv2D(32, kernel_size=(3,3), input_shape=(150,150,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
#chunk-2  
model.add(Conv2D(32, kernel_size=(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
#chunk-3  
model.add(Conv2D(64, kernel_size=(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

In [0]:
#flattenning
model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(2,activation="softmax"))

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        1

In [0]:
# This model is using rmsprop as optimizer instead of SGD
#model.compile(loss=categorical_crossentropy, optimizer="rmsprop", metrics=["acc"])
# will try with sgd = tf.keras.optimizers.SGD(lr=0.1, decay=0.000225, momentum=0.5)
#model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss='categorical_crossentropy',metrics=['accuracy'])
# Cycle -3
model.compile(SGD(lr=0.01), categorical_crossentropy, ["acc"])


In [0]:
#we have to reinitialize dataGenerator to avoid any issue (TBD)
dataGenerator = ImageDataGenerator(rotation_range=40, horizontal_flip=True, zoom_range=0.2, rescale=1/255)


In [29]:
#Create Train Generator and choose the path where the data lies
train_generator = dataGenerator.flow_from_directory("data", target_size=(150,150), batch_size=128)
validation_generator = dataGenerator.flow_from_directory("validation_data", target_size=(150,150),batch_size=128)
 

Found 25000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [30]:
model.fit_generator(train_generator, epochs=10,steps_per_epoch=25000//128,
                    validation_data=validation_generator,validation_steps=(5000/128))
# Since below warning came up so, using Model.fit

#WARNING:tensorflow:From <ipython-input-29-d5df9b1af4c0>:1: Model.fit_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
#Instructions for updating:
#Please use Model.fit, which supports generators.
#model.fit(train_generator, epochs=10)


#model.fit(generator=train_generator,validation_data=validattionGen,epochs=10,validation_split=)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
195/195 [==============================] - 208s 1s/step - loss: 0.6891 - acc: 0.5335 - val_loss: 0.6830 - val_acc: 0.5846
Epoch 2/10
195/195 [==============================] - 204s 1s/step - loss: 0.6814 - acc: 0.5672 - val_loss: 0.6732 - val_acc: 0.6148
Epoch 3/10
195/195 [==============================] - 201s 1s/step - loss: 0.6743 - acc: 0.5820 - val_loss: 0.6666 - val_acc: 0.5904
Epoch 4/10
195/195 [==============================] - 201s 1s/step - loss: 0.6659 - acc: 0.5955 - val_loss: 0.6557 - val_acc: 0.6232
Epoch 5/10
195/195 [==============================] - 201s 1s/step - loss: 0.6616 - acc: 0.6018 - val_loss: 0.6471 - val_acc: 0.6340
Epoch 6/10
195/195 [==============================] - 200s 1s/step - loss: 0.6563 - acc: 0.6076 - val_loss: 0.6447 - val_acc: 0.6300
Epoch 7/10
195/195 [==============================] - 199s 1s/step - loss: 0.6508 - acc: 0.6141 - val_loss: 0.6486 - val_acc: 

In [0]:
#!rm -rf /content/data

In [0]:
#!rm -rf /content/validation_data/

In [0]:
#!rm /content/train.zip

In [0]:
#!rm /content/test1.zip

In [0]:
#!rm -rf /content/train

In [0]:
#!rm /content/test1.zip

In [0]:
#!rm -rf /content/train

In [0]:
#!rm -rf /content/train

In [0]:
#del df2

In [0]:
#!rm -rf /content/test1.zip

In [0]:
#del model